<a href="https://colab.research.google.com/github/jx-dohwan/kluebert_Extract_summary/blob/main/kluebert_Extract_summary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 라이브러리 설치

### Mecab

In [ ]:
!pip3 install konlpy

In [ ]:
!sudo apt-get update

In [ ]:
!sudo apt-get install curl git

In [ ]:
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

### Multiprocess

In [ ]:
!pip3 install multiprocess

### Transformers

In [ ]:
!pip3 install transformers

### PyRouge

In [ ]:
!pip install pyrouge --upgrade
!pip install https://github.com/bheinzerling/pyrouge/archive/master.zip
!pip install pyrouge
!pip show pyrouge
!git clone https://github.com/andersjo/pyrouge.git
from pyrouge import Rouge155
!pyrouge_set_rouge_path '/content/pyrouge/tools/ROUGE-1.5.5'

In [ ]:
!sudo apt-get install libxml-parser-perl

In [ ]:
%%shell
cd pyrouge/tools/ROUGE-1.5.5/data
rm WordNet-2.0.exc.db # only if exist
cd WordNet-2.0-Exceptions
rm WordNet-2.0.exc.db # only if exist

./buildExeptionDB.pl . exc WordNet-2.0.exc.db
cd ../
ln -s WordNet-2.0-Exceptions/WordNet-2.0.exc.db WordNet-2.0.exc.db

### TensorboardX

In [ ]:
!pip3 install tensorboardX

### Copy Source Codes

In [ ]:
!cp -r /content/drive/MyDrive/Fast_Campus/ExSum/SRC .

## 라이브러리 Import 

In [12]:
import os
import json
from tqdm import tqdm
import re
import easydict

## 데이터


###원본 데이터 탐색
- 데이터 구성
  - 원문 데이터 40만 건 (신문기사 30만 건, 기고문 6만 건, 잡지기사 1만 건, 법원 판결문 3만 건)을 활용하여 각각 추출요약 40만 건, 생성요약 40만 건, 총 80만 건의 요약문 도출
  - 원문으로부터 변형 없이 그대로 선택된 3개 문장으로 추출요약문 생성
  - 원문의 내용을 바탕으로 재작성된 생성요약문 생성

In [13]:
DATAPATH = '/content/drive/MyDrive/인공지능/추출요약/data/raw_data/train'
filenames = [x for x in os.listdir (DATAPATH) if x.endswith('json')]
filenames.sort()
filenames

['train_original_law.json',
 'train_original_news.json',
 'train_original_opinion.json']

In [14]:
file = filenames[0]
filelocation = os.path.join(DATAPATH, file)

with open(filelocation, 'r') as json_file:
  data = json.load(json_file) ['documents']

In [15]:
data[0]

{'id': '100004',
 'category': '일반행정',
 'size': 'small',
 'char_count': 377,
 'publish_date': '19841226',
 'title': '부당노동행위구제재심판정취소',
 'text': [[{'index': 0,
    'sentence': '원고가 소속회사의 노동조합에서 분규가 발생하자 노조활동을 구실로 정상적인 근무를 해태하고,',
    'highlight_indices': ''},
   {'index': 1, 'sentence': '노조조합장이 사임한 경우,', 'highlight_indices': ''},
   {'index': 2,
    'sentence': '노동조합규약에 동 조합장의 직무를 대행할 자를 규정해 두고 있음에도 원고 자신이 주동하여 노조자치수습대책위원회를 구성하여 그 위원장으로 피선되어 근무시간중에도 노조활동을 벌여 운수업체인 소속회사의 업무에 지장을 초래하고',
    'highlight_indices': '8,9;68,69'},
   {'index': 3,
    'sentence': '종업원들에게도 나쁜 영향을 끼쳐 소속회사가 취업규칙을 위반하고',
    'highlight_indices': ''},
   {'index': 4,
    'sentence': '고의로 회사업무능률을 저해하였으며 회사업무상의 지휘명령에 위반하였음을 이유로 원고를 징계해고 하였다면,',
    'highlight_indices': '0,3'},
   {'index': 5,
    'sentence': '이는 원고의 노동조합 활동과는 관계없이 회사취업규칙에 의하여 사내질서를 유지하기 위한 사용자 고유의 징계권에 기하여 이루어진 정당한 징계권의 행사로 보아야 한다.',
    'highlight_indices': '17,21'}]],
 'annotator_id': 3783,
 'document_quality_scores': {'readable': 3,
  'accurate': 3,
 

### 원본 데이터에서 필요한 값 추출
- text와 extractive 추출
  - text : 정규식을 이용해 sentence - highlight_indices 사이의 문장 추출한 후 리스트로 저장
  - extracive : 3줄 요약에 해당하는 문장 index 3개가 저장된 리스트
- 신문기사, 기고문, 법원 판결분 3개로 나누어져 잇는 파일을 하나로 합침
  - train, valid 각각에 대해 수행

#### Train 데이터

In [24]:
DATAPATH = '/content/drive/MyDrive/인공지능/추출요약/data/raw_data/train'
filenames = [x for x in os.listdir (DATAPATH) if x.endswith('json')]
filenames.sort()
filenames

['train_original_law.json',
 'train_original_news.json',
 'train_original_opinion.json']

In [25]:
list_dic = []

for file in filenames:
  filelocation = os.path.join(DATAPATH, file)

  with open(filelocation, 'r') as json_file:
    data = json.load(json_file)['documents']

    for x in tqdm (range(len(data))):
      text = data[x]['text']
      text = str(text).replace('"', "'")

      extractive = data[x]['extractive']
      for index, value in enumerate(extractive):
        if value == None:
          extractive[index] = 0

      p = re.compile('(?<=sentence\'\: \')(.*?)(?=\'highlight_indices)')
      texts = p.findall(text)

      sentences = []
      for t in texts:
        sentence = t[:-3]
        sentences.append(sentence)

      mydict = {}
      mydict['text'] = sentences
      mydict['extractive'] = extractive
      list_dic.append(mydict)

100%|██████████| 56760/56760 [00:09<00:00, 6300.66it/s]


In [26]:
len(list_dic)

325072

In [29]:
with open("/content/drive/MyDrive/인공지능/추출요약/data/raw_data/train.json", 'w') as fh:
  json.dump(list_dic, fh)

In [28]:
# def list_chunk(lst, n):
#     return [lst[i:i+n] for i in range(0, len(lst), n)]

# data_chunked = list_chunk(data, 32507) ## 전체 데이터를 10개로 분할

# for i, d in enumerate(data_chunked):
#   with open("/content/drive/MyDrive/인공지능/추출요약/data/raw_data/train.{}.json".format(i), 'w') as fh:
#     json.dump(d, fh)

#### val 

In [30]:
DATAPATH = '/content/drive/MyDrive/인공지능/추출요약/data/raw_data/valid'
filenames = [x for x in os.listdir (DATAPATH) if x.endswith('json')]
filenames.sort()
filenames

['valid_original_law.json',
 'valid_original_news.json',
 'valid_original_opinion.json']

In [31]:
list_dic = []

for file in filenames:
  filelocation = os.path.join(DATAPATH, file)

  with open(filelocation, 'r') as json_file:
    data = json.load(json_file)['documents']

    for x in tqdm (range(len(data))):
      text = data[x]['text']
      text = str(text).replace('"', "'")

      extractive = data[x]['extractive']
      for index, value in enumerate(extractive):
        if value == None:
          extractive[index] = 0

      p = re.compile('(?<=sentence\'\: \')(.*?)(?=\'highlight_indices)')
      texts = p.findall(text)

      sentences = []
      for t in texts:
        sentence = t[:-3]
        sentences.append(sentence)

      mydict = {}
      mydict['text'] = sentences
      mydict['extractive'] = extractive
      list_dic.append(mydict)

100%|██████████| 7008/7008 [00:01<00:00, 4894.16it/s]


In [32]:
with open("/content/drive/MyDrive/인공지능/추출요약/data/raw_data/valid.json", 'w') as fh:
  json.dump(list_dic, fh)